# Look for plate sequences in MMA directory

In [1]:
%pylab notebook
%matplotlib notebook

import os, glob
import csv

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from astropy.table import Table, Column

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


**Note on output of this script:**

The print statements in this script are shut down, by commenting them out. This is so because the tables they output are very long, and having many tables like that in a single running script can overwhelm the browser and/or the localhost server. The main result tables at the end will print by default. If you want to examine intermediate results, feel free to uncomment the print statements, minding that the script may eventually slow down or even crash.



In [2]:
def convert_string(str_value, invalid_value=None, default_result=''):
    return str_value if str_value != invalid_value else default_result   

## Table input and formatting

Some rows in the CSV file have errors and/or missing data. They are discarded, but listed at the end of this cell.

In [3]:
# Build initial table from CSV file

csvfile = open('MMA_directory.csv', 'r')
dict_reader = csv.DictReader(csvfile)

table = Table(names=dict_reader.fieldnames, 
              dtype=(np.int32,'f4','f4','f4','f4', 'S2', 'S2', 'S2', 'S2'))

for row in dict_reader:
    
    try:    
        plate_number = row[dict_reader.fieldnames[0]]
        ra_h = convert_string(row[dict_reader.fieldnames[1]], invalid_value='', default_result=np.NaN)
        ra_m = convert_string(row[dict_reader.fieldnames[2]], invalid_value='', default_result=np.NaN)

        # the dec field has some format inconsistencies
        dec_d = row[dict_reader.fieldnames[3]]
        if dec_d == '':
            dec_d = np.NaN
        elif dec_d is not None and ',' in dec_d:
            dec_d = float(dec_d.split(',')[0]) + float(dec_d.split(',')[0]) / 60.
        elif dec_d is not None and ';' in dec_d:
            dec_d = float(dec_d.split(';')[0]) + float(dec_d.split(';')[0]) / 60.
        else:
            dec_d = float(dec_d)

        exp_min  = convert_string(row[dict_reader.fieldnames[4]], invalid_value='', default_result=np.NaN)
        date     = convert_string(row[dict_reader.fieldnames[5]])
        jd       = convert_string(row[dict_reader.fieldnames[6]])
        emulsion = convert_string(row[dict_reader.fieldnames[7]])
        notes    = convert_string(row[dict_reader.fieldnames[8]])
    
        table.add_row((plate_number, ra_h, ra_m, dec_d, exp_min, date, jd, emulsion, notes))

    except (ValueError, TypeError) as e:
        print(e)
        print(row)
        print()


float() argument must be a string or a real number, not 'NoneType'
{'Plate#': '2471 4 52 26 6 3 38 11 20', 'RA-h': None, 'RA-m': None, 'DEC-d': None, 'Exp-min': None, 'Date': None, 'J.D.': None, 'Emulsion': None, 'Notes': None}

Unable to insert row because of exception in column 'Exp-min':
could not convert string to float: ' unknown'
{'Plate#': '295', 'RA-h': '12', 'RA-m': '38', 'DEC-d': '13.7', 'Exp-min': ' unknown', 'Date': '16_10_27', 'J.D.': '', 'Emulsion': '', 'Notes': 'ep.1855'}

float() argument must be a string or a real number, not 'NoneType'
{'Plate#': '4096 14 2 40 20 63 05 19', 'RA-h': None, 'RA-m': None, 'DEC-d': None, 'Exp-min': None, 'Date': None, 'J.D.': None, 'Emulsion': None, 'Notes': None}

Unable to insert row because of exception in column 'Exp-min':
could not convert string to float: ' 30?'
{'Plate#': '7168', 'RA-h': '18', 'RA-m': '0', 'DEC-d': '-24', 'Exp-min': ' 30?', 'Date': '83_09_26', 'J.D.': '45603.494', 'Emulsion': '103aO', 'Notes': None}

float() argumen

In [4]:
# table.show_in_notebook(display_length=10)

In [5]:
# Split date field into separate columns for year, month, day 

year_values  = []
month_values = []
day_values   = []

for row in table:
    date_str = row['Date']
    date_elements = date_str.split('_')
    
    if len(date_elements) == 3:
        year_values.append(float(date_elements[0]))
        month_values.append(float(date_elements[1]))
        day_values.append(float(date_elements[2]))
    else: 
        year_values.append(0.)
        month_values.append(0.)
        day_values.append(0.)

year_col  = Column(name='Year',  data=year_values)
month_col = Column(name='Month', data=month_values)
day_col   = Column(name='Day',   data=day_values)
    
table.add_columns([year_col, month_col, day_col])

In [6]:
# Add a decimal RA field, to be consistent with the DEC-d field

ra_values = (table['RA-h'] + table['RA-m'] / 60.) * 15.

ra_col = Column(name='RA-d', data=ra_values)
table.add_columns([ra_col])

In [7]:
# table.show_in_notebook(display_length=10)

## Table manipulation

Initially, sort table by year, month, and day, so we can see the time sequence of all plates. Also, discard anything more recent than 1957.

In [8]:
# Sort by year, and hierarchically by month and day
table.sort(['Year', 'Month', 'Day'])

# Select years below 1957
mask = (table['Year'] > 0) & (table['Year'] < 57)
table = table[mask]

In [9]:
table.show_in_notebook(display_length=10)

idx,Plate#,RA-h,RA-m,DEC-d,Exp-min,Date,J.D.,Emulsion,Notes,Year,Month,Day,RA-d
0,4,0.0,0.0,90.0,1.0,13_11_21,,,Pole,13.0,11.0,21.0,0.0
1,7,0.0,9.0,0.0,5.0,13_11_22,,,,13.0,11.0,22.0,2.25
2,9,23.0,24.0,0.1,0.1,13_12_06,,,,13.0,12.0,6.0,351.0
3,12,20.0,26.0,-8.9,11.0,14_07_20,,,,14.0,7.0,20.0,306.5
4,13,20.0,27.0,-8.9,4.0,14_07_20,,,,14.0,7.0,20.0,306.75
5,16,0.0,0.0,90.0,5.0,14_07_22,,,Pole,14.0,7.0,22.0,0.0
6,15,20.0,23.0,-8.5,10.0,14_07_22,,,,14.0,7.0,22.0,305.75
7,14,0.0,0.0,90.0,5.0,14_07_22,,,Pole,14.0,7.0,22.0,0.0
8,17,0.0,0.0,90.0,5.0,14_08_01,,,Pole,14.0,8.0,1.0,0.0
9,18,0.0,0.0,90.0,10.0,14_08_01,,,Pole,14.0,8.0,1.0,0.0


Sorting by coordinates should segregate together all plates for any given telescope pointing.

In [10]:
table.sort(['RA-d', 'DEC-d'])

In [11]:
# table.show_in_notebook(display_length=10)

## Find usable exposure sequences

The goal here is to isolate the plate sequences that share a common pointing and a common date. 

We do that by first collecting together all exposures in any given pointing (we call these 'fields') and then collecting together all exposures in each field that share a common date.

The final product should be a collection of 'sequences', where each sequence includes the plate numbers for any given field and date.

### Segregate exposures by field (pointing)

We start by segregating together all exposures that share a common pointing.

- start with the original table, sorted hierarchically in RA and DEC, as done in the previous step.
- compute the distance, in degrees, between the center of each plate, and the center of the previous plate.
- flag all rows where this distance is below a certain threshold. Flag also the row immediately before, because
it is the reference pointing from which a small distance resulted, thus it belongs to the same field. The flag should be the field sequence number. Note that this assumes a table sorted in RA, DEC.

In [12]:
table.sort(['RA-d', 'DEC-d'])

# Compute distance between plate centers and store in table column
diff_ra  = table['RA-d'][1:]  - table['RA-d'][:-1]
diff_dec = table['DEC-d'][1:] - table['DEC-d'][:-1]

diff = sqrt(diff_ra**2 + diff_dec**2)
table['diff-pos'] = np.insert(diff, 0, np.nan)

# Mark all rows pertaining to a given field, that is, sharing same coordinates, 
# with a field identifier in the form of a sequence number 
fields = []
threshold = 1.5   # make this 1.5 degree for now
# field numbers start from 1. Zero is reserved for rows not associated with any field
field_number = 1
field_number_updated = False

for row in table:
    diff_pos = row['diff-pos']
    if diff_pos < threshold:
        # this row belongs to a field
        fields.append(field_number)
        
        # make sure previous row also belongs to the same field. This is useful only when
        # the row being flagged is the first. It means that the previous row is also in the same
        # field.
        fields[-2] = field_number
        
        field_number_updated = False
    else:
        # this row does not belong to a field. It also signals that the next row that
        # finds itself in a field, should be assigned to a new field, so update field number
        fields.append(0)

        # make sure field number is updated only when valid fields are found
        if not field_number_updated:       
            field_number += 1
            field_number_updated = True
    
field_col = Column(name='Field', data=fields)
table.add_columns([field_col])

table.sort(['Field', 'Year', 'Month', 'Day'])

In [13]:
# table.show_in_notebook(display_length=10)

### Sanity check for field segregation algorithm

Check every row marked with a zero field that it indeed has no match in any of the fields. Some rows are left behind by the procedure above. Any row found that way, has its 'Field' updated with the correct value.

In [14]:
# separate rows with zero field from rows within a valid field
mask = (table['Field'] == 0)
table_zero = table[mask]

mask = (table['Field'] > 0)
table_non_zero = table[mask]

In [15]:
for row_zero, row_original in zip(table_zero, table):
    ra_zero = row_zero['RA-d']
    dec_zero = row_zero['DEC-d']
    for row_non_zero in table_non_zero:
        ra_non_zero = row_non_zero['RA-d']
        dec_non_zero = row_non_zero['DEC-d']
        
        diff_ra  = ra_zero - ra_non_zero
        diff_dec = dec_zero - dec_non_zero

        diff = sqrt(diff_ra**2 + diff_dec**2)

        # when an unflagged row is found belonging to an existing field, it's added there
        if diff < threshold:
            row_original['Field'] = row_non_zero['Field']

In [16]:
# table.sort(['Field', 'Year', 'Month', 'Day'])
# table.show_in_notebook(display_length=10)

### Segregate exposures in each field based on date


Now, we look into each field for exposures made on the same date.

- sort table on Year, Month, Day
- for each row, compute time interval in days that separates it from previous row
- flag all rows where this interval is zero. Flag also the row immediately before, because it is the reference date from which a zero interval resulted, thus it belongs to the same date. The flag should be the date sequence number. Note that this assumes a table sorted in YMD.


In [17]:
table.sort(['Year', 'Month', 'Day'])

diff = (abs(table['Year'][1:] - table['Year'][:-1]))*365 + (abs(table['Month'][1:] - table['Month'][:-1]))*30 + \
       abs(table['Day'][1:] - table['Day'][:-1])
table['diff-date'] = np.insert(diff, 0, np.nan)

# Mark all rows pertaining to a given date with a date identifier in the form of a sequence number 
dateseq = []
# dateseq numbers start from 1. 
dateseq_number = 1
dateseq_number_updated = False

for row in table:
    diff_date = row['diff-date']
    if diff_date < 1:
        # this row belongs to a date
        dateseq.append(dateseq_number)
        
        # make sure previous row also belongs to the same date. This is useful only when
        # the row being flagged is the first. It means that the previous row has also the same
        # date.
        dateseq[-2] = dateseq_number
        
        dateseq_number_updated = False
    else:
        # this row does not belong to a date. It also signals that the next row that
        # finds itself in a date, should be assigned to a new date, so update dateseq
        dateseq.append(0)

        # make sure dateseq number is updated only when valid dates are found
        if not dateseq_number_updated:       
            dateseq_number += 1
            dateseq_number_updated = True
    
dateseq_col = Column(name='Date Seq.', data=dateseq)
table.add_columns([dateseq_col])

table.sort(['Date Seq.', 'Year', 'Month', 'Day'])

In [18]:
# table.show_in_notebook(display_length=10)

### Sanity check for date segregation

We use the JD column to check the reliability of assigninf exposures to any given date. Only about 17% of the plates have assigned JD values. 

We isolate these rows in a separate table and examine the Date Seq. assignments against JD values.

In [19]:
mask = (table['J.D.'] != '')
table_jd = table[mask]

table_jd.sort(['Date Seq.', 'J.D.'])

In [20]:
# table_jd.show_in_notebook(display_length=10)

Examining visually, we can see that blocks of rows with the same sequence number share the same integer part of JD, as expected. 

Analysis of the fractional part of JD should enable us to have some information about the time separation between multiple exposures in a sequence.

### Segregate by both field and date

We have to look for rows that share the same field and date sequence numbers. 

We build a column that concatenates both numbers in a single string, then we can sort that and use a similar segregation procedure as used above to identify fields and date sequence numbers. 

To keep the process clean, lets remove all rows for which either Field or Date Seq. is zero.

In [21]:
mask = (table['Field'] != 0) 
table1 = table[mask]
mask = (table1['Date Seq.'] != 0) 
table = table1[mask]

In [22]:
# table.show_in_notebook(display_length=10)

In [23]:
# Build a Field-Date column that encodes both Field and Date information for each plate
table.sort(['Field', 'Date Seq.'])

table['Field'] = Column(table['Field'].astype(str), dtype='str') 
table['Date Seq.'] = Column(table['Date Seq.'].astype(str), dtype='str') 

# combine columns
array_shape = table['Field'].data.shape
fill_string = "-"
delimiter = np.full(array_shape, fill_string, dtype='U10')
data1 = np.char.add(table['Field'].data, delimiter)
data  = np.char.add(data1, table['Date Seq.'].data)

# column might exist already; happens when working interatively
fielddate_col = Column(name='Field-Date', data=data)
try:
    table.add_columns([fielddate_col])
except ValueError:
    table.replace_column('Field-Date', fielddate_col)
    
table.sort(['Year', 'Month', 'Day'])

In [24]:
# table.show_in_notebook(display_length=10)

Now, we look into the Field-Date column, looking for plates that share the same field-date value.

- sort table on YMD columns
- for each row, compare its Field-Date value with the preceding row (use boolean column to store the result; it has the same role as the diff-pos and diff-date columns)
- flag all rows where this flag is True. Also, mark the row immediately before, because it is the reference field-date from which a match resulted, thus it belongs to the same field-date sequence. The mark should be the 'Sequence' sequential number. 



In [25]:
table.sort(['Year', 'Month', 'Day'])

isSequence = table['Field-Date'][1:] == table['Field-Date'][:-1] 

table['Is-in-Seq'] = np.insert(isSequence, 0, False)

In [26]:
# table.show_in_notebook(display_length=10)

In [27]:
# Mark all rows pertaining to a given sequence 
seqseq = []
# seqseq numbers start from 1. 
seqseq_number = 1
seqseq_number_updated = False

for row in table:
    isinseq = row['Is-in-Seq']
    if isinseq:
        # this row belongs to a sequence
        seqseq.append(seqseq_number)
        
        # make sure previous row also belongs to the same sequence. This is useful only when
        # the row being flagged is the first. It means that the previous row has also the same
        # sequence number.
        seqseq[-2] = seqseq_number
        
        seqseq_number_updated = False
    else:
        # this row does not belong to a sequence. It also signals that the next row that
        # finds itself in a sequence, should be assigned to a new sequence, so update seqseq
        seqseq.append(0)

        # make sure seqseq number is updated only when valid sequences are found
        if not seqseq_number_updated:       
            seqseq_number += 1
            seqseq_number_updated = True
    
seqseq_col = Column(name='Sequence', data=seqseq)
table.add_columns([seqseq_col])

table.sort(['Sequence'])

In [28]:
# table.show_in_notebook(display_length=10)

### Sanity check for sequence assignation

Same as sanity check for field assignation.

In [29]:
# separate rows with zero sequence number from rows within a valid sequence number
mask = (table['Sequence'] == 0)
table_zero = table[mask]

mask = (table['Sequence'] > 0)
table_non_zero = table[mask]

In [30]:
for row_zero, row_original in zip(table_zero, table):
    seq_zero = row_zero['Field-Date']
    for row_non_zero in table_non_zero:
        seq_non_zero = row_non_zero['Field-Date']
        
        is_match = seq_zero == seq_non_zero

        # when an unflagged row is found belonging to an existing sequence, 
        # flag it as belonging to that sequence
        if is_match:
            row_original['Field-Date'] = row_non_zero['Field-Date']          

Use JD values to verify that plate numbers in any given sequence, are time-ordered. (Fields 183 and 185 are exceptions)

In [31]:
table1 = table['Plate#', 'RA-h', 'RA-m', 'DEC-d', 'Date', 'J.D.', 'Sequence']
mask = table1['Sequence'] > 0
table1 = table1[mask]
table1.sort(['Sequence', 'Plate#'])

In [32]:
# table1.show_in_notebook(display_length=10)

## Results

- dates overlapping the POSS-I date range do not contain sequences (they end Dec 1947).
- 

In [33]:
# Build lists of plates in each sequence
seqnumber_list = np.unique(table1['Sequence'].data)

rows = []

for seqnumber in seqnumber_list:
    
    mask = (table1['Sequence'] == seqnumber)
    t = table1[mask]
    
    plateno = t['Plate#']

    # these are defined from the first row for this sequence
    rah  = t['RA-h'][0]
    rad  = t['RA-m'][0]
    dec  = t['DEC-d'][0]
    date = t['Date'][0]
    
    pns = []
    for pn in plateno:
        pns.append(pn)
        
    nplates = len(pns)
        
    rows.append([seqnumber, nplates, rah, rad, dec, date, pns])
    
result = Table(names=['Sequence', '# of plates', 'RA-h', 'RA-m', 'DEC-d', 'Date', 'Plates'], rows=rows)

In [34]:
result.show_in_notebook(display_length=10)

idx,Sequence,# of plates,RA-h,RA-m,DEC-d,Date,Plates
0,2,2,20.0,26.0,-8.9,14_07_20,"[12, 13]"
1,3,2,0.0,0.0,90.0,14_07_22,"[14, 16]"
2,4,2,0.0,0.0,90.0,14_08_01,"[17, 18]"
3,5,2,0.0,0.0,90.0,14_08_13,"[21, 22]"
4,6,2,23.0,53.0,23.9,14_09_13,"[31, 32]"
5,7,3,10.0,14.0,49.6,14_09_17,"[43, 44, 421]"
6,8,4,23.0,42.0,24.4,14_09_19,"[48, 49, 50, 51]"
7,9,4,23.0,38.0,24.5,14_09_21,"[53, 54, 55, 56]"
8,10,2,23.0,48.0,-11.9,14_09_22,"[57, 58]"
9,11,2,22.0,50.0,22.1,14_10_20,"[61, 62]"
